In [1]:
### Library functions
import numpy as np
import torch 
import torch.nn as nn
import torch.optim as optim
import networkx as nx
import matplotlib.pyplot as plt
import random 
from matplotlib import animation
import networkx.algorithms.centrality as nx_centrality
from collections import deque
from matplotlib import animation
import math

# Dataset
## Training dataset
The global parameters of the generate_random_graph function:
1. FAKE_DIFF_ITER: the fake nodes diffusion iteration
2. FAKE_SEED_NUM: the initial fake seed set number
3. NODE_NUM: the graph nodes number
4. EDGE_NUM: the graph edges number 
5. M_INDEX: the generate graph method index

In [2]:
def generate_random_graph(difusion_iteration, method_index, fake_seed_num, num_nodes, num_edges, probability = 0.5,m = 2, radius = None, k_nearest_neighbor = None, degree = None, seed = None):
    random_graph = nx.Graph()
    if method_index == 0:
        random_graph = nx.gnm_random_graph(num_nodes, num_edges)

    elif method_index == 1:
        random_graph = nx.erdos_renyi_graph(num_nodes, probability)

    elif method_index == 2:
         # m means the edges number from the create point to existing point
        random_graph = nx.barabasi_albert_graph(num_nodes, m)

    elif method_index == 3:
        random_graph = nx.watts_strogatz_graph(num_nodes, k_nearest_neighbor, probability)
    for (u,v) in random_graph.edges:
        if FIXED == True:
            random_graph.edges[u, v]["weight"] = PROBABILITY
        else:
            random_graph.edges[u, v]["weight"] =  random.uniform(0,1)
    for node in random_graph.nodes():
        random_graph.nodes[node]['state'] = 0 # 0初始状态；1:true；2:fake
    node_number = random_graph.number_of_nodes()
    start_node_index = [0] *fake_seed_num
    index_range = range(0,  node_number)
    fake_seed_set = random.sample(index_range, fake_seed_num)
    fake_active_nodes = fake_seed_set.copy()
    fake_nodes_set = fake_seed_set.copy()
    for _ in range(difusion_iteration):
        temp_fake = []
        for v in fake_active_nodes:
            for nbr in random_graph.neighbors(v): 
                if random_graph.nodes[nbr]['state'] == 0 : 
                    edge_data = random_graph.get_edge_data(v, nbr)
                    if random.uniform(0, 1) < edge_data['weight']:
                        fake_nodes_set.append(nbr)
                        temp_fake.append(nbr)
                        random_graph.nodes[nbr]['state'] = 2
        fake_active_nodes = temp_fake.copy()
        temp_fake.clear()    
    for node in fake_nodes_set:
        random_graph.nodes[node]['state'] = 2
        
    return random_graph,fake_active_nodes,fake_seed_set,fake_nodes_set


## Create and normalize test data

In [3]:
import scanpy as sc 
import scipy as sp
import scipy.sparse  # call as sp.sparse
import scipy.io as sio

# filename_ca_HepTh = r"D:\course (Kou Hari)\2023 semaster 1\COMP5703 Capstone\data\ca-HepTh.mtx"
# filename_ca_GrQc = r"D:\course (Kou Hari)\2023 semaster 1\COMP5703 Capstone\data\ca-GrQc.mtx"
# filename_tech_p2p_gnutella = r"D:\course (Kou Hari)\2023 semaster 1\COMP5703 Capstone\data\tech-p2p-gnutella.mtx"
filename_ca_HepTh = 'ca-HepTh.mtx'
filename_ca_GrQc = 'ca-GrQc.mtx'
filename_tech_p2p_gnutella = 'tech-p2p-gnutella.mtx'

In [4]:
def read_graph(filename):
    
#     adata = sc.read(filename)
#     sparse_matrix = sp.load_npz(filename)
    data = sio.mmread(filename)
    G = nx.from_scipy_sparse_array(data)
#     G = nx.from_scipy_sparse_array(adata.X, create_using=nx.MultiGraph)
    print("node number:",G.number_of_nodes())
    print("edge number:",G.number_of_edges())
    return G, G.number_of_nodes(), G.number_of_edges()

In [5]:
G_ca_HepTh,N_ca_HepTh,E_ca_HepTh= read_graph(filename_ca_HepTh)

node number: 9877
edge number: 25998


In [6]:
G_ca_GrQc,N_ca_GrQc,E_ca_GrQc = read_graph(filename_ca_GrQc)

node number: 5242
edge number: 14496


In [7]:
G_tech_p2p_gnutella,N_tech_p2p_gnutella,E_tech_p2p_gnutella = read_graph(filename_tech_p2p_gnutella)

node number: 62561
edge number: 147878


In [8]:
def normalization_graph(graph):
    graph_node_number = graph.number_of_nodes()
    graph_edge_number = graph.number_of_edges()
    for node in graph.nodes():
        graph.nodes[node]['state'] = 0
    for (u,v) in graph.edges():
        graph.edges[u,v]['weight'] = random.uniform(0,1)
    return graph,graph_node_number,graph_edge_number

In [9]:
def fake_nodes_diffusion(graph,fake_seed_num,iteration):
    node_number = graph.number_of_nodes()
    start_node_index = [0] *fake_seed_num
    index_range = range(0,  node_number)
    fake_seed_set = random.sample(index_range, fake_seed_num)
    fake_active_nodes = fake_seed_set.copy()
    fake_nodes_set = fake_seed_set.copy()
    for node in fake_seed_set:
        graph.nodes[node]['state'] = 2
    #fake news diffusion
    for _ in range(iteration):
        temp_fake = []
        for v in fake_active_nodes:
            for nbr in graph.neighbors(v): 
                if graph.nodes[nbr]['state'] == 0 : 
                    edge_data = graph.get_edge_data(v, nbr)
                    if random.uniform(0, 1) < edge_data['weight']:
                        fake_nodes_set.append(nbr)
                        temp_fake.append(nbr)
                        graph.nodes[nbr]['state'] = 2
        fake_active_nodes = temp_fake.copy()  
    for node in fake_nodes_set:
        random_graph.nodes[node]['state'] = 2
    return graph,fake_active_nodes

In [10]:
def statistic_draw_graph(Graph,draw = False):
    color_list = []
    fake = 0
    true = 0
    normal = 0
    for node in Graph.nodes():
        if Graph.nodes[node]['state'] == 0:
            normal += 1
            color_list.append('blue')
        elif Graph.nodes[node]['state'] == 1:
            true += 1
            color_list.append('green')
        elif Graph.nodes[node]['state'] == 2:
            fake += 1
            color_list.append('red')
    print("normal nodes number:",normal)
    print("true nodes number:",true)
    print("fake nodes number:",fake)
    print("the edge of graph:",Graph.number_of_edges())
    #nx.draw(Graph, node_color= color_list) 
    if draw:
        nx.draw_circular(Graph, node_color= color_list)
        plt.show()

# Environment Class
The global parameters in the Environment Class:
1. ITERATION: control the diffusion times during the true and fake nodes antagonistic process
2. MAX_STEP: control the max step of this epoch
3. SEED_SIZE: the initial size of the seed set, the initial parameters of Env class.

The input variable of the Env() class:
1. graph: the training graph or testing graph
2. seed_size: SEED_SIZE;the initial size of the seed set
3. fake_set: the initial fake

The functions in the Env class:
1. reset: reset the parameters
2. step: the experiment excuation progress
3. select_initial_seeds: select the initial seed randomly
4. select_initial_seeds_rules: select the intial seed set according to different criterion
5. add_seed: add the seed nodes according to the action number from the agent
6. get_state: get the current state of the graph
7. diffusion_process: the procees of the true and fake nodes antagonistic
8. get_image: draw the image of the graph

In [11]:
class Env():
    def __init__(self, graph,seed_size,fake_set):
        self.graph_initial = graph
        self.seed_size = seed_size
        self.seed_set = []
        self.fake_set = fake_set
        self.state = None
        self.reset()

    def reset(self):
        #select the seed set:
        self.graph = self.graph_initial.copy()
        self.step_count = 0
        self.add_number = 0
        self.seed_set, action = self.select_initial_seeds_rules(self.graph,self.seed_size)
        self.true_active_nodes = self.seed_set.copy()
        self.fake_active_nodes = self.fake_set.copy()
        self.state = self.get_state()
        return self.state, action
    
    def step(self,action):
        count_state0 = 0
        for node in self.graph.nodes():
            if self.graph.nodes[node]['state'] == 0:
                count_state0 += 1
        if action > 0 and (len(self.seed_set) + ADD_SEED_NUMBER <= self.graph.number_of_nodes()) and count_state0 >= ADD_SEED_NUMBER:
            self.add_number += 1
            centrality_methods = ["None", "random", "degree", "closeness", "betweenness", "eigenvector"]
            centrality_method = centrality_methods[action]
            self.seed_set = self.add_seed(self.graph,self.seed_set,action,ADD_SEED_NUMBER)
            if(self.true_active_nodes[-ADD_SEED_NUMBER:]!= self.seed_set[-ADD_SEED_NUMBER:]):
                self.true_active_nodes.extend(self.seed_set[-ADD_SEED_NUMBER:])
        self.graph,self.true_active_nodes,self.fake_active_nodes = self.diffusion_process(self.graph,
                                                                                     self.true_active_nodes,
                                                                               self.fake_active_nodes,ITERATION)
        #print("current image:")
        #self.get_image(self.graph)
        
        # terminal condition                                                                    
        all_true_nodes = []
        all_fake_nodes = []
        normal_nodes= []                                                                             
        for node in self.graph.nodes():
            if self.graph.nodes[node]['state'] == 1:
                all_true_nodes.append(node)
            elif self.graph.nodes[node]['state'] == 2:
                all_fake_nodes.append(node)
            elif self.graph.nodes[node]['state'] == 0:
                normal_nodes.append(node)
        number_nodes = len(self.graph.nodes())
        number_true_nodes = len(all_true_nodes) 
        number_fake_nodes = len(all_fake_nodes)
        # terminal condition: over the MAX_STEP / less 1% nodes are normal nodes
        terminated = bool(self.step_count >= MAX_STEP 
                        or len(normal_nodes)<=0.1 * number_nodes) 
        penatly = math.pow(1.05,self.add_number)
        if not terminated:
            if len(all_true_nodes)>len(all_fake_nodes):
                reward = (len(all_true_nodes)  - len(all_fake_nodes) - penatly + 5)/(len(all_true_nodes) + len(all_fake_nodes))
            else:
                reward = (len(all_true_nodes)  - len(all_fake_nodes) - penatly)/(len(all_true_nodes) + len(all_fake_nodes))
            done = False
            self.step_count +=1
        else: 
            # when this iteration is end, give a little big reward.
            reward = 30
            done = True
            
#             self.step_count >= MAX_STEP: 
#             # when this iteration is end, give a little big reward.
#             reward = 20
#             done = True
#         elif len(normal_nodes)<=0.01 * number_nodes:
#             reward = 30
#             done = True
            
#         print(reward)
        self.state = self.get_state()
        return self.state, reward ,done 
    # select the initial seed set randomly
    def select_initial_seeds(self, graph, seed_number):
        node_number = graph.number_of_nodes()
        start_node_index = [0] *seed_number
        index_range = range(0,  node_number)
        start_node_index = random.sample(index_range, seed_number)
        for node in graph.nodes():
            if node in start_node_index:
                graph.nodes[node]['state'] = 1
        return start_node_index
    # select the initial seed set according to different rules
    def select_initial_seeds_rules(self,graph,seed_number):
        # 1 random 2 "degree",3 "closeness", 4"betweenness", 5"eigenvector"
        action = random.randint(0,5)
        start_node_index = []
        centrality_scores = []
        if action == 0:   
            node_number = graph.number_of_nodes()
            start_node_index = [0] *seed_number
            index_range = range(0,  node_number)
            start_node_index = random.sample(index_range, seed_number)
        else:
            if action == 1:
                centrality_scores  = list(nx_centrality.degree_centrality(self.graph).values())
            elif action == 2:
                centrality_scores  = list(nx_centrality.degree_centrality(self.graph).values())
            elif action == 3:
                centrality_scores = list(nx_centrality.closeness_centrality(self.graph).values())
            elif action == 4:
                centrality_scores = list(nx_centrality.betweenness_centrality(self.graph).values())
            elif action == 5:
                centrality_scores = list(nx_centrality.eigenvector_centrality(self.graph).values())
            for _ in range(seed_number):
                max_number = max(centrality_scores)
                index = centrality_scores.index(max_number)
                while(graph.nodes[index]['state'] != 0):
                    centrality_scores[index] = -1
                    max_number = max(centrality_scores)
                    index = centrality_scores.index(max_number)  
                start_node_index.append(index)
                centrality_scores[index] = -1
        for node in graph.nodes():
            if node in start_node_index:
                graph.nodes[node]['state'] = 1
        return start_node_index,action
        
    def add_seed(self,graph,seed_set,action,add_number):
        # 1 random 2 "degree",3 "closeness", 4"betweenness", 5"eigenvector"
        new_seed_set = []
        new_seed_set = seed_set.copy()
        centrality_scores = []
        state0 = 0
        for node in graph.nodes():
            if graph.nodes[node]['state'] == 0:
                state0 += 1
        if state0 >= ADD_SEED_NUMBER:
            if action == 1:
                node_number = graph.number_of_nodes()
                for _ in range(ADD_SEED_NUMBER):
                    index = random.randint(0,node_number-1)
                    while (index in seed_set) or (graph.nodes[index]['state'] in [1,2])  :
                        index = random.randint(0,node_number-1)
                    new_seed_set.append(index)
                    graph.nodes[index]['state']  = 1

            else: 
                if action == 2:
                    centrality_scores  = list(nx_centrality.degree_centrality(self.graph).values())
                elif action == 3:
                    centrality_scores = list(nx_centrality.closeness_centrality(self.graph).values())
                elif action == 4:
                    centrality_scores = list(nx_centrality.betweenness_centrality(self.graph).values())
                elif action == 5:
                    centrality_scores = list(nx_centrality.eigenvector_centrality(self.graph).values())
                for node in seed_set:
                    centrality_scores[node] = -1
                for _ in range(ADD_SEED_NUMBER):
                    index = centrality_scores.index(max(centrality_scores))
                    while(graph.nodes[index]['state'] != 0):
                        index = centrality_scores.index(max(centrality_scores))
                        # print(index)
                        centrality_scores[index] = -1
                    # print("add_seed, index:",index)
                    new_seed_set.append(index)
                    graph.nodes[index]['state'] = 1
                    centrality_scores[index] = -1
        return new_seed_set
    def get_state(self):
        node_state = []
        for node in self.graph.nodes():
            node_state.append(self.graph.nodes[node]['state'])
        return nx.to_numpy_array(self.graph), len(self.seed_set), node_state
    
    def diffusion_process(self, G ,true_active_set,fake_active_set,itertaion):
        G = self.graph.copy()
        true_active_nodes = true_active_set.copy()
        fake_active_nodes = fake_active_set.copy()
        # print("true_active_nodes begin:",true_active_nodes)
        # print("fake_active_nodes begin:",fake_active_nodes)
        for _ in range(itertaion):
            tmp_true_nodes ={}
            for v in true_active_nodes: 
                for nbr in G.neighbors(v):
                    if G.nodes[nbr]['state'] in [0]:
                        edge_data = G.get_edge_data(v, nbr)
                        random_possibility = random.uniform(0, 1)
                        if random_possibility < edge_data['weight']:
                            G.nodes[nbr]['state'] == 3
                            tmp_true_nodes.update({nbr:random_possibility})
                            true_active_set.append(nbr)
                    else:
                        continue
            tmp_fake_nodes ={}
            for v in fake_active_nodes:
                for nbr in G.neighbors(v):
                    if G.nodes[nbr]['state'] in [0,3]:
                        edge_data = G.get_edge_data(v, nbr)
                        random_possibility = random.uniform(0, 1)
                        if random_possibility < edge_data['weight']:
                            tmp_fake_nodes.update({nbr:random_possibility})
                            fake_active_set.append(nbr)
                            G.nodes[nbr]['state'] == 4
            true_active_nodes = list(tmp_true_nodes.keys())
            fake_active_nodes = list(tmp_fake_nodes.keys())
            for node in tmp_true_nodes.keys():
                if node in tmp_fake_nodes.keys() :
                    if tmp_true_nodes.get(node) >= tmp_fake_nodes.get(node):
                        fake_active_set.remove(node)
                        fake_active_nodes.remove(node)
                    elif tmp_true_nodes.get(node) < tmp_fake_nodes.get(node):
                        true_active_set.remove(node)
                        true_active_nodes.remove(node)
                else:
                    continue
            for node in true_active_set:
                G.nodes[node]['state'] = 1
            for node in fake_active_set:
                G.nodes[node]['state'] = 2
            # print("true_active_nodes",true_active_set)
            # print("fake_active_nodes",fake_active_set)
        return G,true_active_nodes,fake_active_nodes

        
    def get_image(self,Graph):
        color_list = []
        for node in Graph.nodes():
            if Graph.nodes[node]['state'] == 0:
                color_list.append('blue')
            elif Graph.nodes[node]['state'] == 1:
                color_list.append('green')
            elif Graph.nodes[node]['state'] == 2:
                color_list.append('red')
        #nx.draw(Graph, node_color= color_list) 
        nx.draw_circular(Graph, node_color= color_list)
        plt.show()

# Agent

##  Device: GPU/CPU
Note:

if you use Macbook with Apple M1/M2 core, please run the code in the Mac Device;

if you use Windows with CUDA core, please run the code in the Windows Device;

### Mac Device

In [14]:
# # if use MAC(Apple M1/M2) please run the follow codes
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

### Windows Device

In [15]:
import torch
print("CUDA Available:", torch.cuda.is_available())

CUDA Available: True


In [16]:
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name()
    print("GPU Device Name:", device_name)

GPU Device Name: NVIDIA GeForce RTX 3070 Ti Laptop GPU


In [17]:
if torch.backends.cudnn.is_available():
    print("cuDNN Available")

cuDNN Available


In [18]:
# if use CUDA please run the follow codes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## DQN Agent

In [19]:
class DQNAgent:
    def __init__(self, state_dim, action_dim, learning_rate, gamma, epsilon, epsilon_decay):
        self.q_network = QNetwork(state_dim, action_dim).to(device)
        self.target_network = QNetwork(state_dim, action_dim).to(device)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        self.loss_fn = nn.MSELoss()
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.memory = deque(maxlen=10000)

    def get_action(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.randint(0, action_dim)
        else:
            state = torch.tensor(state, dtype=torch.float32).to(device)
            q_values = self.q_network(state)
#           q_values = self.q_network(torch.tensor(state, dtype=torch.float32))
            return torch.argmax(q_values.cpu()).item()

    def remember(self, state, action, reward, next_state, done):
        #state = torch.tensor(state).float().unsqueeze(0)  # ensure state is a tensor
        self.memory.append((state, action, reward, next_state, done))

    def train(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state, done in minibatch:
            #print("next_state:", next_state)
            #state = torch.tensor(state, dtype=torch.float32).to(device)
            state_tensor = state_to_tensor(state).to(device)
            #print(state_tensor.shape)
            #print(self.q_network(state_tensor).shape)
            action_index = torch.tensor(action, dtype=torch.long)
            target = self.q_network(state_tensor)[0, action_index]
            #target = self.q_network(state_tensor)[action]
            if done:
                target_value = reward
            else:
                next_state_tensor = state_to_tensor(next_state).to(device)
                #next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
                next_q_values = self.target_network(next_state_tensor).detach()
                target_value = reward + self.gamma * torch.max(next_q_values).item()

            #loss = self.loss_fn(target, torch.tensor(float(target_value)))
            loss = self.loss_fn(target, torch.tensor(float(target_value)).to(device))
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

        self.epsilon *= self.epsilon_decay

    def update_target_network(self):
        self.target_network.load_state_dict(self.q_network.state_dict())

    def save(self, path):
        torch.save(self.q_network.state_dict(), path)

    def load(self, path):
        self.q_network.load_state_dict(torch.load(path))
        self.target_network.load_state_dict(self.q_network.state_dict())

In [20]:
def state_to_tensor(state):
    graph, num_seed_nodes, node_state = state
    
    graph_tensor = torch.tensor(graph, dtype=torch.float32).view(-1).unsqueeze(0)
    num_seed_nodes_tensor = torch.tensor([num_seed_nodes], dtype=torch.float32).unsqueeze(0)
    node_state_tensor = torch.tensor(node_state, dtype=torch.float32).unsqueeze(0)
    
#     state_tensor = torch.cat((graph_tensor, num_seed_nodes_tensor, node_state_tensor), dim=1)
    state_tensor = torch.cat((num_seed_nodes_tensor, node_state_tensor), dim=1)
    
    return state_tensor

In [21]:
class QNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)# (2500 + 50 + 2) (10000 + 100 + 2)
        
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training Function
The input valiable of train_dqn function:

1. agent: the initialized Agent class
2. env: the initialized Env class
3. batch_size: BATCH_SIZE
4. update_target_every: UPDATE;How many iterations to update the network
5. dqn_agent_name: the name of DQN agent, the file extension is '.pth'
6. brenchmark_action: Start benchmark comparison and specifying an action(1-5)
7. Demo: Whether state should be displayed or saved each time

In [33]:
def train_dqn(agent, env, episodes, batch_size, update_target_every,dqn_agent_name  = None,brenchmark_action = None,Demo = False):
    state_history= []
    rewards_list = []
    for episode in range(episodes):
        state,init_action = env.reset()
        if brenchmark_action is not None:
            init_action = brenchmark_action
        #print(state)
#         state = torch.tensor(state).float().unsqueeze(0)  # ensure state is a tensor
        total_reward = 0
        done = False

        while not done:
            state_tensor = state_to_tensor(state)
            #print(state_tensor)
            if brenchmark_action is not None:
                action = brenchmark_action
            else:
                action = agent.get_action(state_tensor)
            next_state, reward, done = env.step(action)
            #print(next_state)
            agent.remember(state, action, reward, next_state, done)
            agent.train(batch_size)
            total_reward += reward
            state = next_state
            if Demo:
                state_history.append(state)  # Add current state to state_history
        if not Demo:
            state_history.append(state)
        rewards_list.append(total_reward)
        if episode % update_target_every == 0:
            agent.update_target_network()

        print(f"Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")
    if dqn_agent_name is not None:
        torch.save(agent,dqn_agent_name)
    return state_history, rewards_list

## Test Function

In [23]:
def test_dqn_model(model, env, episodes):
    rewards_list = []
    for episode in range(episodes):
        state,init_action = env.reset()
        total_reward = 0
        done = False

        while not done:
            state_tensor = state_to_tensor(state)
            action = model.get_action(state_tensor)
            next_state, reward, done = env.step(action)
            total_reward += reward
            state = next_state
            
        rewards_list.append(total_reward)

        print(f"Test Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")
        
    return rewards_list

In [24]:
def test_dqn(agent, env, episodes):
    rewards_list = []

    for episode in range(episodes):
        state,init_action = env.reset()
        total_reward = 0
        done = False

        while not done:
            state_tensor = state_to_tensor(state)
            action = agent.get_action(state_tensor)
            next_state, reward, done = env.step(action)
            total_reward += reward
            state = next_state
            
        rewards_list.append(total_reward)

        print(f"Test Episode {episode + 1}/{episodes}, Total Reward: {total_reward}")
        
    return rewards_list

## Experiment

In [25]:
# the Global parameter of generate_random_graph
FAKE_DIFF_ITER = 3 # the fake nodes diffusion iteration
FAKE_SEED_NUM = 5 # the initial fake seed set number
NODE_NUM = N_ca_GrQc # the intial graph nodes number
EDGE_NUM = E_ca_GrQc # the intial graph edges number
M_INDEX = 0 # the generate graph method index

# the Global parameter of Env class
ITERATION = 1 #control the diffusion times during the true and fake nodes antagonistic process
MAX_STEP = 40 # control the max step of this epoch
SEED_SIZE = 10 # the initial size of the seed set, the initial parameters of Env class.
ADD_SEED_NUMBER = 1 # the number of nodes that are added to seed set

# the Global parameter of Agentclass
state_dim = NODE_NUM + 1 #state_dim: Dimension of state
action_dim = 6 #action_dim:  Dimension of action space
LR = 0.001#learning_rate: 
GAMMA = 0.99 # gamma:
EPSILON = 1.0# epsilon: 
EPSILON_DECAY = 0.995 # epsilon_decay: 
# the Global parameter of train and test function
batch_size = 128 #batch_size: 
UPDATE = 10 #update_target_every: 
ALL_EPISODES = 1 # total number of iteration times

FIXED = False

In [26]:
N_ca_GrQc,E_ca_GrQc

(5242, 14496)

In [27]:
intial_train_graph_ca_GrQc,active_fake_set_ca_GrQc,intial_fake_set_ca_GrQc,all_fake_nodes_ca_GrQc = generate_random_graph(FAKE_DIFF_ITER,M_INDEX,FAKE_SEED_NUM,NODE_NUM, EDGE_NUM )

In [28]:
statistic_draw_graph(intial_train_graph_ca_GrQc)

normal nodes number: 5143
true nodes number: 0
fake nodes number: 99
the edge of graph: 14496


In [29]:
SEED_SIZE = len(active_fake_set_ca_GrQc)

In [30]:
env_ca_GrQc = Env(intial_train_graph_ca_GrQc, SEED_SIZE, active_fake_set_ca_GrQc)

In [31]:
agent_ca_GrQc = DQNAgent(state_dim, action_dim, LR , GAMMA, EPSILON, EPSILON_DECAY)

## Training

In [ ]:
state_history_ca_GrQc, rewards_list_ca_GrQc = train_dqn(agent_ca_GrQc, env_ca_GrQc, ALL_EPISODES, batch_size ,UPDATE,dqn_agent_name ="ca_GrQc.pth")

## Testing

In [ ]:
G_ca_GrQc,N_ca_GrQc, E_ca_GrQc = normalization_graph(G_ca_GrQc)

In [ ]:
env_ca_GrQc = Env(G_ca_GrQc, SEED_SIZE, active_fake_set_ca_GrQc)

In [ ]:
model = torch.load("ca_GrQc.pth")
test_rewards_list_ca_GrQc = test_dqn_model(model, env_ca_GrQc, 2) 

In [ ]:
import pandas as pd

# Create a DataFrame from the list
data = pd.DataFrame({'Rewards_test_ca_GrQc': test_rewards_list_ca_GrQc,'Rewards_train_ca_GrQc': rewards_list_ca_GrQc})

# Save the DataFrame to an Excel file
data.to_excel('test_rewards_ca_GrQc.xlsx', index=False)